## Multiple Contracts

To demonstrate how we can use multiple contracts, we will create a strategy that looks at overnight price gaps for 2 contracts
It chooses the name that has the largest percentage overnight gap and goes long that name. The strategy exits by the end of the day
If price moves against us, we use a stop loss to get out. We have 15 minute bars of Coke (KO) and Pepsi (PEP) stock prices

In [16]:
# %%checkall
import pandas as pd
import pyqstrat as pq
import numpy as np
from collections import defaultdict

contract_names = ['ko', 'pep']

filenames = {'ko': './support/coke_15_min_prices.csv.gz', 'pep': './support/pepsi_15_min_prices.csv.gz'}

on_rets: dict[np.datetime64, tuple[str, float, np.array, np.array]] = {}
for name in contract_names:
    prices = pd.read_csv(filenames[name], usecols=['date', 'c'])
    prices.columns = ['timestamp', 'c']
    prices['date'] = prices.timestamp.values.astype('M8[D]')
    prices[f'on_ret'] = np.where(prices.date > prices.date.shift(1), prices.c  / prices.c.shift(1) - 1, np.nan)
    date_rets = prices[np.isfinite(prices.on_ret) & (prices.on_ret > 0)]
    dates = date_rets.date.values.astype('M8[D]')
    on_ret = date_rets.on_ret.values
    c = date_rets.c.values
    for i, date in enumerate(dates):
        if date not in on_rets or on_ret[i] > on_rets[date][1]:
            date_prices = prices[prices.date == date]
            on_rets[date] = (name, on_ret[i], date_prices.timestamp.values, date_prices.c.values)
    
price_dict: dict[str, dict[np.datetime64, float]] = defaultdict(dict)

for _, (name, _, timestamps, prices) in on_rets.items():
    price_dict[name].update({timestamps[i]: prices[i] for i in range(len(timestamps))})
    



# def get_overnight_returns(filename: str) -> dict[np.datetime64, tuple[float, float]]:
#     '''Read 15 minute price bars'''
#     prices = pd.read_csv(filename)[['date', 'c']]
#     prices.columns = ['timestamp', 'c']
#     prices['date'] = prices.timestamp.values.astype('M8[D]')
#     prices[f'on_ret'] = np.where(prices.date > prices.date.shift(1), prices.c  / prices.c.shift(1) - 1, np.nan)
#     prices = prices[np.isfinite(prices.on_ret)]
#     c = prices.c.values
#     dates = prices.date.values.astype('M8[D]')
#     on_rets = prices.on_ret.values
#     on_rets_dict = {dates[i]: (on_rets[i], c) for i in range(len(dates)) if on_rets[i] > 0}
#     return on_rets_dict

# on_rets: dict[str, dict[np.datetime64, tuple[float, float]]] = {}
# for name in contract_names:
#     print(filenames[name])
#     on_rets[name] = get_overnight_returns(filenames[name])
    
# dates = np.unique([date for on_ret in on_rets.values() for date in on_ret])
# entry_names: dict[np.datetime64, tuple[str, float, float]] = {date: ('', 0., np.nan) for date in dates}
# for name, rets in on_rets.items():
#     for date in dates:
#         if date in rets and rets[date][0] > entry_names[date][1]:
#             entry_names[date] = (name, rets[date])
            
# # get stop price for each date
# # for date, (name, ret) in entry_names:
# #     start_price = entry_names[date][]

        
    
    


# # # merge the 2 dataframes together so we have a separate column for each indicator and signal
# # prices['date'] = prices.timestamp.values.astype('M8[D]')
# # on_rets = pd.merge(coke['date', 'c']], pepsi[[, on=['timestamp'], how='outer', suffixes=['_ko', '_pep'])


# # for name in contract_names:
# #     price_col = prices[f'c_{name}']
# #     prices[f'overnight_ret_{name}'] = np.where(prices.date > prices.date.shift(1), price_col  / price_col.shift(1) - 1, np.nan)

# # daily_entries: dict[np.datetime64, str] = {}
# # daily_df = prices[np.isfinite()]

# # # # mark points just before EOD. We enter a marker order at these points so we have one bar to get filled
# # prices['eod'] = np.where(prices.date.shift(-2) > prices.date, True, False)   
    
# # # for each day, pick the greatest overnight return. If no returns are positive, pick no return
# # entry_contracts: dict[np.datetime64, str] = {}
# # overnight_ret_cols = [f'overnight_ret_{name}' for name in contract_names]
# # prices['max_on_ret'] = prices[overnight_ret_cols].max(axis=1)
# # prices['entry_contract'] = prices[overnight_ret_cols].idxmax(axis=1)
# # prices['entry_contract'] = np.where(prices.max_on_ret <= 0, np.nan, prices.entry_contract)






    
    # if the price drops by 0.5% after we enter in the morning take our loss and get out
#     prices[f'stop_prc_{name}'] = np.where(np.isfinite(prices.overnight_ret), price_col * 0.995, np.nan) 
#     prices['stop_price'] = prices.stop_price.fillna(method='ffill')  # fill in the stop price for the rest of the day

# # mark points just before EOD. We enter a marker order at these points so we have one bar to get filled
# prices['eod'] = np.where(prices.date.shift(-2) > prices.date, True, False)   
# prices['stop'] = np.where(prices.c < prices.stop_price, True, False)  # whether we should exit because we are stopped out


# strat_builder = pq.StrategyBuilder(data=prices)
on_rets

{numpy.datetime64('2019-01-15'): ('pep',
  0.004005589194224557,
  array(['2019-01-15 09:45:00', '2019-01-15 10:00:00',
         '2019-01-15 10:15:00', '2019-01-15 10:30:00',
         '2019-01-15 10:45:00', '2019-01-15 11:00:00',
         '2019-01-15 11:15:00', '2019-01-15 11:30:00',
         '2019-01-15 11:45:00', '2019-01-15 12:00:00',
         '2019-01-15 12:15:00', '2019-01-15 12:30:00',
         '2019-01-15 12:45:00', '2019-01-15 13:00:00',
         '2019-01-15 13:15:00', '2019-01-15 13:30:00',
         '2019-01-15 13:45:00', '2019-01-15 14:00:00',
         '2019-01-15 14:15:00', '2019-01-15 14:30:00',
         '2019-01-15 14:45:00', '2019-01-15 15:00:00',
         '2019-01-15 15:15:00', '2019-01-15 15:30:00',
         '2019-01-15 15:45:00', '2019-01-15 16:00:00'], dtype=object),
  array([107.78 , 108.12 , 108.01 , 108.16 , 108.4  , 108.52 , 108.62 ,
         108.66 , 108.67 , 108.72 , 108.86 , 108.95 , 108.81 , 108.66 ,
         108.53 , 108.57 , 108.53 , 108.54 , 108.46 , 108.4 

In [17]:
price_dict

defaultdict(dict,
            {'pep': {'2019-01-15 09:45:00': 107.78,
              '2019-01-15 10:00:00': 108.12,
              '2019-01-15 10:15:00': 108.01,
              '2019-01-15 10:30:00': 108.16,
              '2019-01-15 10:45:00': 108.4,
              '2019-01-15 11:00:00': 108.52,
              '2019-01-15 11:15:00': 108.62,
              '2019-01-15 11:30:00': 108.66,
              '2019-01-15 11:45:00': 108.67,
              '2019-01-15 12:00:00': 108.72,
              '2019-01-15 12:15:00': 108.86,
              '2019-01-15 12:30:00': 108.95,
              '2019-01-15 12:45:00': 108.81,
              '2019-01-15 13:00:00': 108.66,
              '2019-01-15 13:15:00': 108.53,
              '2019-01-15 13:30:00': 108.57,
              '2019-01-15 13:45:00': 108.53,
              '2019-01-15 14:00:00': 108.54,
              '2019-01-15 14:15:00': 108.46,
              '2019-01-15 14:30:00': 108.4,
              '2019-01-15 14:45:00': 108.38,
              '2019-01-15 15:00:

In [12]:
on_rets

{numpy.datetime64('2019-01-15'): ('pep',
  0.004005589194224557,
  array(['2019-01-15 09:45:00', '2019-01-15 10:00:00',
         '2019-01-15 10:15:00', '2019-01-15 10:30:00',
         '2019-01-15 10:45:00', '2019-01-15 11:00:00',
         '2019-01-15 11:15:00', '2019-01-15 11:30:00',
         '2019-01-15 11:45:00', '2019-01-15 12:00:00',
         '2019-01-15 12:15:00', '2019-01-15 12:30:00',
         '2019-01-15 12:45:00', '2019-01-15 13:00:00',
         '2019-01-15 13:15:00', '2019-01-15 13:30:00',
         '2019-01-15 13:45:00', '2019-01-15 14:00:00',
         '2019-01-15 14:15:00', '2019-01-15 14:30:00',
         '2019-01-15 14:45:00', '2019-01-15 15:00:00',
         '2019-01-15 15:15:00', '2019-01-15 15:30:00',
         '2019-01-15 15:45:00', '2019-01-15 16:00:00'], dtype=object),
  array([107.78 , 108.12 , 108.01 , 108.16 , 108.4  , 108.52 , 108.62 ,
         108.66 , 108.67 , 108.72 , 108.86 , 108.95 , 108.81 , 108.66 ,
         108.53 , 108.57 , 108.53 , 108.54 , 108.46 , 108.4 